In [ ]:
from google.colab import drive
drive.mount('./content')

In [1]:
import keras
keras.__version__

'2.4.3'

- 생성자

In [30]:
from keras import layers
import numpy as np

latent_dim = 32
height = 32
width = 32
channels = 3

generator_input = keras.Input(shape=(latent_dim,))

# 입력을 16 × 16 크기의 128개 채널을 가진 특성 맵으로 변환합니다
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.BatchNormalization()(x)      # BN추가 5개
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

# 합성곱 층을 추가합니다
x = layers.Conv2DTranspose(256, 5, padding='same')(x)  # Conv2D
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)

# 32 × 32 크기로 업샘플링합니다
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)

# 합성곱 층을 더 추가합니다
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)

# 32 × 32 크기의 1개 채널을 가진 특성 맵을 생성합니다
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)
generator.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense_2 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
batch_normalization (BatchNo (None, 32768)             131072    
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 32768)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 16, 16, 256)       819456    
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 256)      

- 판별자
    - : 후보이미지(진짜 또는 가짜)를 입력으로 받고 2개의 클래스로 분류
    - 클래스는 '생성된 이미지' 또는 '훈련세트에서 온 진짜 이미지' 이다.


In [41]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Dropout(0.5)(x)       # droput 4개 추가
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Dropout(0.5)(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Dropout(0.5)(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Dropout(0.5)(x)
x = layers.Flatten()(x)

# 드롭아웃 층을 넣는 것이 아주 중요합니다!
x = layers.Dropout(0.4)(x)

# 분류 층
x = layers.Dense(1, activation='sigmoid')(x)

# (32,32, 3) 크기의 입력을 이진분류 결정(진짜/가짜)로 변환하는 판별자 모델 객체 생성
discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()


discriminator_optimizer = keras.optimizers.RMSprop(lr=0.000008  # 학습률 낮춤
                                                   , clipvalue=1.0 # 옵티마이저에서 (값을 지정하여) 그래디언트 클리핑을 사용 
                                                   , decay=1e-8)   # 안정된 훈련을 위해서 학습률 감쇠를 사용합니다
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 30, 30, 128)       0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_19 (LeakyReLU)   (None, 14, 14, 128)       0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 14, 14, 128)     

- 적대적 네트워크
    - 생성자와 판별자를 연결하여 GAN을 설정
    - 훈련 시, 생성자가 판별자를 속이는 방향으로 학습
    - 이 모델은 잠재 공간의 포인트를 "진짜" 또는 "가짜"의 분류 결정으로 변환
    - 훈련에 사용되는 타깃 레이블은 항상 "진짜 이미지"이다.
    - 훈련하는 동안 판별자를 동결하는 것이 중요. (학습되지 않도록. 가중치가 업데이트되지 않도록)


- : GAN을 훈련하는 것은 DISCRIMINATOR가 가짜 이미지를 보고 진짜라고 예측하도록 만들기 위해 GENERATOR의 가중치를 업데이트.



In [42]:
# 판별자의 가중치가 훈련되지 않도록 설정합니다(gan 모델에만 적용됩니다)
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

- 데이터 로드

In [43]:
import os
from keras.preprocessing import image

In [44]:
img_dir = "../input/convert-img"

# img_dir = '/content/content/MyDrive/Colab Notebooks/sai/10img.zip (Unzipped Files)'

In [45]:
from PIL import Image
from numpy import asarray



In [46]:
x_train = []


for i in os.listdir(img_dir):
  # load the image
  image = Image.open(os.path.join(img_dir, i))
  # convert image to numpy array
  data = asarray(image)
  x_train.append(data)

x_train= asarray(x_train)
x_train = x_train/255.    #정규화

In [47]:
x_train= asarray(x_train)

In [48]:
x_train.shape # (504개의 이미지, 32, 32, 3채널)

(504, 32, 32, 3)

In [49]:
# 데이터를 정규화합니다
x_train = x_train.reshape(
    (x_train.shape[0],) + (height, width, channels)).astype('float32') / 255.

- GAN 훈련 구현


- 훈련시, 적대적 손실이 크게 증가한다면(= 판별자의 손실은 0으로 향하는 경우), 판별자의 학습률을 낮추고 판별자의 드롭아웃 비율을 높이기

In [50]:
iterations = 100
batch_size = 64

# 생성된 이미지 저장 위치 지정
save_dir = './gan_images-2/'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

# 훈련 반복 시작
start = 0
for step in range(iterations):
    # 잠재 공간에서 무작위로 포인트를 샘플링합니다
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # 가짜 이미지를 디코딩합니다
    generated_images = generator.predict(random_latent_vectors)

    # 진짜 이미지와 연결합니다
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images]) 

    # 진짜와 가짜 이미지를 구분하여 레이블을 합칩니다
    # generate 1, real 0
    labels = np.concatenate([np.ones((batch_size, 1)),
                             np.zeros((batch_size, 1))])
    # 레이블에 랜덤 노이즈를 추가합니다. 아주 중요합니다!
    labels += 0.05 * np.random.random(labels.shape)

    # discriminator를 훈련합니다
    d_loss = discriminator.train_on_batch(combined_images, labels)

    # 잠재 공간에서 무작위로 포인트를 샘플링합니다
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # 모두 “진짜 이미지"라고 레이블을 만듭니다 (사실은 거짓)
    misleading_targets = np.zeros((batch_size, 1))

    # generator를 훈련합니다(gan 모델에서 discriminator의 가중치는 동결됩니다)
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    if start > len(x_train) - batch_size:
      start = 0

    # 중간 중간 저장하고 그래프를 그립니다 (100번 스텝마다)
    if step % 10 == 0:
        # 모델 가중치를 저장합니다
        gan.save_weights('gan.h5')

        # 측정 지표를 출력합니다
        print('스텝 %s에서 판별자 손실: %s' % (step, d_loss))
        print('스텝 %s에서 적대적 손실: %s' % (step, a_loss))

        # 생성된 이미지 하나를 저장합니다
        # img = image.array_to_img(generated_images[0] * 255., scale=False)
        # img = generated_images[0] * 255.
        img = Image.fromarray(generated_images[0] * 255., 'RGB')
        img.save(os.path.join(save_dir, 'generated_img' + str(step) + '.png'))

        # 비교를 위해 진짜 이미지 하나를 저장합니다
        # img = image.array_to_img(real_images[0] * 255., scale=False)
        # img = real_images[0] * 255.
        img = Image.fromarray(real_images[0] * 255., 'RGB')
        img.save(os.path.join(save_dir, 'real_img' + str(step) + '.png'))
        
        
# real과 generated 이미지가 각각 10개씩 저장됌. gan_images 폴더에. 

스텝 0에서 판별자 손실: 0.6955441832542419
스텝 0에서 적대적 손실: 0.7043163776397705
스텝 10에서 판별자 손실: 0.6394461393356323
스텝 10에서 적대적 손실: 0.7396118640899658
스텝 20에서 판별자 손실: 0.6723966002464294
스텝 20에서 적대적 손실: 0.7357919216156006
스텝 30에서 판별자 손실: 0.6127578020095825
스텝 30에서 적대적 손실: 0.7272346019744873
스텝 40에서 판별자 손실: 0.6448622941970825
스텝 40에서 적대적 손실: 0.7391330599784851
스텝 50에서 판별자 손실: 0.6658486127853394
스텝 50에서 적대적 손실: 0.7557953596115112
스텝 60에서 판별자 손실: 0.6492408514022827
스텝 60에서 적대적 손실: 0.7559599876403809
스텝 70에서 판별자 손실: 0.6843950748443604
스텝 70에서 적대적 손실: 0.7567429542541504
스텝 80에서 판별자 손실: 0.6766461133956909
스텝 80에서 적대적 손실: 0.7347612380981445
스텝 90에서 판별자 손실: 0.6757476329803467
스텝 90에서 적대적 손실: 0.7545153498649597


In [ ]:
'''
import time

iterations = 1000
batch_size = 64
save_dir = './gan_images/'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

# 훈련 반복 시작
start = 0
for step in range(iterations):
    s = time.time()
    # 잠재 공간에서 무작위로 포인트를 샘플링합니다
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # 가짜 이미지를 디코딩합니다
    generated_images = generator.predict(random_latent_vectors)

    # 진짜 이미지와 연결합니다
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])

    # 진짜와 가짜 이미지를 구분하여 레이블을 합칩니다
    labels = np.concatenate([np.ones((batch_size, 1)),
                             np.zeros((batch_size, 1))])
    # 레이블에 랜덤 노이즈를 추가합니다. 아주 중요합니다!
    labels += 0.05 * np.random.random(labels.shape)

    # discriminator를 훈련합니다
    d_loss = discriminator.train_on_batch(combined_images, labels)

    # 잠재 공간에서 무작위로 포인트를 샘플링합니다
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # 모두 “진짜 이미지"라고 레이블을 만듭니다
    misleading_targets = np.zeros((batch_size, 1))

    # generator를 훈련합니다(gan 모델에서 discriminator의 가중치는 동결됩니다)
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    if start > len(x_train) - batch_size:
      start = 0

    # 중간 중간 저장하고 그래프를 그립니다
    if step % 100 == 0:
        # 모델 가중치를 저장합니다
        gan.save_weights('gan.h5')

        # 측정 지표를 출력합니다
        print('스텝 %s에서 판별자 손실: %s' % (step, d_loss))
        print('스텝 %s에서 적대적 손실: %s' % (step, a_loss))

        # 생성된 이미지 하나를 저장합니다
        # img = image.array_to_img(generated_images[0] * 255., scale=False)
        # img = generated_images[0] * 255.
        img = Image.fromarray(generated_images[0] * 255., 'RGB')
        img.save(os.path.join(save_dir, 'generated_img' + str(step) + '.png'))

        # 비교를 위해 진짜 이미지 하나를 저장합니다
        # img = image.array_to_img(real_images[0] * 255., scale=False)
        # img = real_images[0] * 255.
        img = Image.fromarray(real_images[0] * 255., 'RGB')
        img.save(os.path.join(save_dir, 'real_img' + str(step) + '.png'))
    e = time.time()
    print('%d번째 소요시간: ' %step ,end='')
    print(e-s)

'''

In [13]:
import matplotlib.pyplot as plt

In [53]:
# 잠재 공간에서 랜덤한 포인트를 샘플링합니다
random_latent_vectors = np.random.normal(size=(10, latent_dim))

# 가짜 이미지로 디코딩합니다
generated_images = generator.predict(random_latent_vectors)
generated_images




array([[[[ 1.08919904e-01, -8.65708962e-02,  8.51807371e-02],
         [ 7.32825845e-02, -5.11350073e-02,  1.06641263e-01],
         [ 1.11892708e-01,  4.52628173e-03, -8.44608154e-03],
         ...,
         [-2.00575590e-03, -4.33255136e-02,  4.63296659e-02],
         [ 1.15833348e-02,  1.85447698e-03, -2.97548473e-02],
         [-2.92662457e-02, -3.08919756e-04, -3.97513099e-02]],

        [[-2.74372771e-02, -1.27698168e-01,  6.04342632e-02],
         [ 1.40196294e-01, -4.39131446e-03,  6.82982802e-02],
         [ 5.15903831e-02,  7.16285631e-02,  1.96350124e-02],
         ...,
         [-2.55922731e-02, -2.68872492e-02, -2.72063427e-02],
         [-3.84704359e-02,  1.82997510e-02, -6.56344891e-02],
         [-3.97326499e-02,  2.20478866e-02, -9.21749696e-02]],

        [[ 6.10084161e-02, -1.37171715e-01,  1.39644369e-01],
         [-5.45327552e-04, -1.04225524e-01,  1.46072343e-01],
         [-2.86261179e-02,  3.54590304e-02, -3.55748273e-02],
         ...,
         [-1.33655041e-0

In [ ]:
'''
# 잠재 공간에서 랜덤한 포인트를 샘플링합니다
random_latent_vectors = np.random.normal(size=(10, latent_dim))

# 가짜 이미지로 디코딩합니다
generated_images = generator.predict(random_latent_vectors)

for i in range(generated_images.shape[0]):
    img = image.array_to_img(generated_images[i] * 255., scale=False)
    plt.figure()
    plt.imshow(img)
    
plt.show()
'''

In [19]:

gan_img_dir = "./gan_images"

for i in os.listdir(gan_img_dir):
  # load the image
    img = Image.open(os.path.join(gan_img_dir, i))
    img = asarray(img)
    img = image.array_to_img(img[i] * 255., scale=False)
    plt.figure()
    plt.imshow(img)
    
plt.show()

AttributeError: 'JpegImageFile' object has no attribute 'array_to_img'

In [ ]:
# 잠재 공간에서 랜덤한 포인트를 샘플링합니다
random_latent_vectors = np.random.normal(size=(10, latent_dim))

# 가짜 이미지로 디코딩합니다
generated_images = generator.predict(random_latent_vectors)

for i in range(generated_images.shape[0]):
    img = image.array_to_img(generated_images[i] * 255., scale=False)
    plt.figure()
    plt.imshow(img)
    
plt.show()

In [26]:
generated_img = []


gan_img_dir = "./gan_images"

for i in os.listdir(gan_img_dir):
  # load the image
  image = Image.open(os.path.join(gan_img_dir, i))
  # convert image to numpy array
  data = asarray(image)
  generated_img.append(data)

generated_img= asarray(generated_img)
generated_img = generated_img/255.    #정규화
generated_img

array([[[[0.95294118, 0.94901961, 0.94901961],
         [0.24313725, 0.96078431, 0.95686275],
         [0.95686275, 0.24313725, 0.89803922],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.50588235],
         [0.50196078, 0.        , 0.23529412]],

        [[0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.50588235],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.50588235],
         [0.50196078, 0.        , 0.23529412]],

        [[0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],

        ...,

        [[0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]

In [29]:
generated_img[0]

array([[[0.95294118, 0.94901961, 0.94901961],
        [0.24313725, 0.96078431, 0.95686275],
        [0.95686275, 0.24313725, 0.89803922],
        ...,
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.50588235],
        [0.50196078, 0.        , 0.23529412]],

       [[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.50588235],
        ...,
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.50588235],
        [0.50196078, 0.        , 0.23529412]],

       [[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ]],

       ...,

       [[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0

In [28]:
x_train[0]

array([[[0.00036909, 0.0005075 , 0.00046136],
        [0.00035371, 0.00049212, 0.00044598],
        [0.00035371, 0.00049212, 0.00044598],
        ...,
        [0.00035371, 0.00052288, 0.00046136],
        [0.00035371, 0.00052288, 0.00046136],
        [0.00033833, 0.0005075 , 0.00044598]],

       [[0.00035371, 0.00049212, 0.00044598],
        [0.00035371, 0.00052288, 0.00046136],
        [0.00035371, 0.00052288, 0.00046136],
        ...,
        [0.00035371, 0.00052288, 0.00046136],
        [0.00035371, 0.00052288, 0.00046136],
        [0.00035371, 0.00052288, 0.00046136]],

       [[0.00035371, 0.00052288, 0.00046136],
        [0.00035371, 0.00052288, 0.00046136],
        [0.00035371, 0.00052288, 0.00046136],
        ...,
        [0.00035371, 0.00052288, 0.00046136],
        [0.00035371, 0.00052288, 0.00046136],
        [0.00035371, 0.00052288, 0.00046136]],

       ...,

       [[0.00035371, 0.00052288, 0.00046136],
        [0.00035371, 0.00052288, 0.00046136],
        [0.00035371, 0